In [5]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta


In [7]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta

col = ['date', 'rate', 'review_text', 'object_address']
path = '/Users/camille/repo/Hetic/projet_gouv/scraping/Data/google_reviews.csv'
data =pd.read_csv(path, usecols=col)

data['date'] = data['date'].replace('a year ago', '1 year ago')
data['date'] = data['date'].replace('a month ago','1 month ago')
data['date'] = data['date'].replace('a week ago','1 week ago')

current_datetime = datetime.now()
data['date'] = data['date'].apply(lambda x: current_datetime - relativedelta(years=int(x.split()[0])) if 'year' in x
                                     else current_datetime - relativedelta(months=int(x.split()[0])) if 'month' in x
                                     else current_datetime - relativedelta(weeks=int(x.split()[0])) if 'week' in x
                                     else current_datetime - relativedelta(days=int(x.split()[0])) if 'day' in x
                                     else None)



# Extraire uniquement la partie de la date
data['date'] = data['date'].apply(lambda x: x.date())


data['date'] = pd.to_datetime(data['date'])
loc = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/Data/location.csv', sep=';')
loc.rename(columns={'adresse':'object_address'}, inplace=True)
loc['object_address'] = loc['object_address'].str.lower().str.strip()
data['object_address'] = data['object_address'].str.lower().str.strip()
#remove comma
data['object_address'] = data['object_address'].str.replace(',', '')
data['object_address'] = data['object_address'].str.replace('11 rue pelée 75011 paris', '11 rue pelee 75011 paris')
data['object_address'] = data['object_address'].str.replace('3 bd diderot 75012 paris', '3 boulevard diderot 75012 paris')
data['object_address'] = data['object_address'].str.replace('75 av. jean jaurès 75019 paris', '75 avenue jean jaures 75019 paris')
data['object_address'] = data['object_address'].str.replace('78 bd ney 75018 paris', '78 boulevard ney 75018 paris')
lieux = data['object_address'].unique()
address_to_lieu = loc.set_index('object_address')['name'].to_dict()

# Ajouter la colonne "name" au DataFrame location en utilisant la correspondance de object_address
data['object_address'] = data['object_address'].map(address_to_lieu)
data.drop(data[data['object_address'] == 'ƒadam'].index, inplace=True)
df_lieu = data.groupby(['object_address']).agg(count_rate=('rate', 'count'), mean_rate=('rate', 'mean')).reset_index()



In [11]:
loc = pd.read_csv('/Users/camille/repo/Hetic/projet_gouv/scraping/Data/location.csv', sep=';')
loc.rename(columns={'adresse':'object_address'}, inplace=True)
loc['object_address'] = loc['object_address'].str.lower().str.strip()
data['object_address'] = data['object_address'].str.lower().str.strip()
#remove comma
data['object_address'] = data['object_address'].str.replace(',', '')

In [15]:
data['object_address'] = data['object_address'].str.replace('11 rue pelée 75011 paris', '11 rue pelee 75011 paris')
data['object_address'] = data['object_address'].str.replace('3 bd diderot 75012 paris', '3 boulevard diderot 75012 paris')
data['object_address'] = data['object_address'].str.replace('75 av. jean jaurès 75019 paris', '75 avenue jean jaures 75019 paris')
data['object_address'] = data['object_address'].str.replace('78 bd ney 75018 paris', '78 boulevard ney 75018 paris')

In [16]:
lieux = data['object_address'].unique()

In [18]:
address_to_lieu = loc.set_index('object_address')['name'].to_dict()

# Ajouter la colonne "name" au DataFrame location en utilisant la correspondance de object_address
data['object_address'] = data['object_address'].map(address_to_lieu)

In [20]:
data.drop(data[data['object_address'] == 'ƒadam'].index, inplace=True)


In [22]:
df_lieu = data.groupby(['object_address']).agg(count_rate=('rate', 'count'), mean_rate=('rate', 'mean')).reset_index()


In [27]:
top = df_lieu.sort_values(by='mean_rate', ascending=False).head(1)


In [25]:
min = df_lieu.sort_values(by='mean_rate', ascending=True).head(1)


In [29]:
min['object_address'].values[0]


'Pole emploi Diderot'